In [1]:
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def dydx(x,y):
    
    #set derivatives 
    
    #equation is d^2y/dx^2 = -y
    
    #we then write 
    #dydx = z
    #dzdx = -y
    
    #set y = y[0]
    #set z = y[1]
    
    #declare an array
    y_derivs = np.zeros(2)
    
    #set dydx = z
    y_derivs[0] = y[1]
    
    #set dzdx = -y
    y_derivs[1] = -1*y[0]
    
    #here we return an array
    return y_derivs

In [15]:
def rk4(deriv,tRange,t_step,x0,v0):
    
    t0 = tRange[0]
    tf = tRange[2*np.pi]
    resultsX = []
    resultsV = []
    resultsT = []
    
    x = x0
    v = v0
    t = t0
    
    resultsX.append(x)
    resultsV.append(v)
    resultsT.append(t)
    
    while t < tf:
        
        x = x
        v = v
        # Full Time Step: Saves this one!
        k1 = dt*deriv(x, v)
        k2 = dt*deriv(x + k1[0]/2.0,v + k1[1]/2.0)
        k3 = dt*deriv(x + k2[0]/2.0, v + k2[1]/2.0)
        k4 = dt*deriv(x + k3[1], v + k3[1])
        x += k1[0]/6. + k2[0]/3. + k3[0]/3. + k4[0]/6.
        v += k1[1]/6. + k2[1]/3. + k3[1]/3. + k4[1]/6.
        resultsX.append(x)
        resultsV.append(v)
        resultsT.append(t)
        
    return resultsX, resultsY

## Define adaptive step size driver for RK

In [16]:
def rk4_ad(dydx,x_i,y_i,nv,h,tol):
    
    #define safety scale
    SAFETY    = 0.9
    H_NEW_FAC = 2.0
    
    #set max number of iterations
    i_max = 10000
    
    #set iteration variable
    i = 0
    
    #create an error
    Delta = np.full(nv,2*tol)
    
    #remember the step
    h_step = h
    
    #adjust step
    while(Delta.max()/tol > 1.0):
        
        #estimate error by taking one step of size h
        #vs two steps of size h/2
        y_2  = rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        y_1  = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_1,nv,0.5*h_step)
        
        #compute an error
        Delta = np.fabs(y_2 - y_11)
        
        #if error is too large, take a smaller step
        if(Delta.max()/tol>1.0):
            
            #our error is too large, decrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        #check iteration
        if(i>=i_max):
            print("Too many iterations in rk4_mv_ad()")
            raise StopIteration("Ending after i = ",i)
            
        #iterate
        i += 1
        
        
    #next time, try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we actually took
    return y_2, h_new, h_step



In [19]:
def rk4_mv(dydx,a,b,y_a,tol):
    
    #deriv is the derivative wrt x
    #a is the lower bound
    #b is the upper bound
    #y_a are boundary conditions
    #tol is the tolerance for integrating y
    
    #define our starting step
    xi = a
    yi = y_a.copy()
    
    #an initial step size == make very small
    h = 1.0e-4
    
    #set max number of iterations
    i_max = 10000
    
    #set iteration variable
    i = 0
    
    #set number of coupled odes to the size of y_a
    nv = len(y_a)
    
    
    #set initial conditions
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    #set flag
    flag = 1
    
    #loop until we reach the right side 
    while(flag):
        
        #calculate y_i+1
        yi_new, h_new, h_step = rk4_mv(dydx,xi,yi,nv,h,tol)
        
        #update step
        h = h_new
        
        #prevent an overshoot
        if(xi+h_step>b):
            
            #take a smaller step 
            h = b-xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = rk4_mv(dydx,xi,yi,nv,h,tol)
            
            #break
            flag = 0
            
            
        #update values 
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]          #yi contins values of y and z
        del y
        y = y_new
        
        #prevent too many iterations
        if(i>=i_max):
            
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number = ",i)
            
            
        #iterate
        i += 1
        
        #output some information
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i,xi,h_step,b)
        print(s)
        
        #break if new xi is == b
        if(xi==b):
            flag = 0
            
    #return the answer
    return x,y



## Perform integration

In [20]:
a = 0.0
b = 2.0*np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

#perform integration
x,y = rk4_mv(dydx,a,b,y_0,tolerance)

TypeError: rk4_mv() takes 5 positional arguments but 6 were given